# 필요 모듈 임포트

In [254]:
import re
import pandas as pd
import numpy as np
from konlpy.tag import Kkma
import re
import unicodedata
import re
import nltk
from string import whitespace, punctuation
from nltk.tokenize import sent_tokenize
from hanspell import spell_checker
from pykospacing import Spacing
from tqdm import tqdm
from kss import split_sentences
import warnings

tqdm.pandas()
warnings.filterwarnings('ignore')

In [255]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/parkjunhyeong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 데이터 로드

In [256]:
#기간별 SNS 데이터 로드 및 병합
load_path = './'
twitter_df1 = pd.read_excel(load_path + 'Analysis_SnsData_20190501_20210930_(청년정책)&&~(rt__독취사)_BY_ARGOS.xlsx', sheet_name = '트위터', skiprows = 1)
twitter_df2 = pd.read_excel(load_path + 'Analysis_SnsData_20211001_20230626_(청년정책)&&~(rt__독취사)_BY_ARGOS.xlsx', sheet_name = '트위터', skiprows = 1)
twitter_df = pd.concat([twitter_df1, twitter_df2], ignore_index = True)

In [257]:
twitter_df

,작성일,작성자,내용
0,2020/09/16,BehonBot,[ 분야별·지역별 #청년정책 ]\n\n▶ 일자리·취업창업 청년정책 http://20...
1,2020/06/13,Fkvkdpffk,@Two36198438 @EjayKim1 신보라 청년정책조정위원히 합류반대\n한다....
2,2020/06/13,NzCjqjiRnfwkZxc,@Two36198438 @jinyeongkim2000 신보라 청년정책조정위원히 합류...
3,2020/06/13,FQdFY39LoL7bY8O,@Two36198438 신보라 청년정책조정위원히 합류반대\n한다.....정세균 비상...
4,2020/06/13,s17221,@Two36198438 신보라 청년정책조정위원히 합류반대\n한다.....정세균 비상...
...,...,...,...
7549,2021/10/05,moefkorea,"<2022년도 예산안, 청년정책 톺아보기!>\n2022년 예산안이 공개되었습니다. ..."
7550,2021/10/05,Korea_land,놓치면 완전 손해!\n쳥넌 주거정책 다 모아봤-집!??\n⠀\n조건에 해당된다면\n...
7551,2021/10/04,drewstar___,"오늘 신문 요약/ 청년 없는 청년 정책, 달에서 벌어지는 미국/중국 우주패권 정책,..."
7552,2021/10/01,BreakNewsCom,"예천군, &quot;청년이 공감하는 청년정책&quot;을 위해 다양한 활동 펼친다...."


# 전처리

In [258]:
#내용에 링크나 사진을 포함하는 게시물들 제거하기 (기사나 광고성 게시물들 제거)
twitter_df_n = twitter_df[~twitter_df['내용'].str.contains('http')].reset_index(drop = True)
twitter_df_n.drop_duplicates(subset = '내용', inplace = True, ignore_index = True)
twitter_df_n

,작성일,작성자,내용
0,2020/06/13,Fkvkdpffk,@Two36198438 @EjayKim1 신보라 청년정책조정위원히 합류반대\n한다....
1,2020/06/13,NzCjqjiRnfwkZxc,@Two36198438 @jinyeongkim2000 신보라 청년정책조정위원히 합류...
2,2020/06/13,FQdFY39LoL7bY8O,@Two36198438 신보라 청년정책조정위원히 합류반대\n한다.....정세균 비상...
3,2020/06/12,Two36198438,신보라 청년정책조정위원히 합류반대\n한다.....정세균 비상식적인 인간을\n 어떻...
4,2021/09/15,union1030,"청년유니온은 2012년 서울시와 사회적교섭으로, 2013년 청년일자리 정책협약을 맺..."
...,...,...,...
2305,2021/10/10,disillusioneder,작년과 올해 20대 여성 자살자 비율이 폭발적으로 늘어났고(동세대 남자는 줄어듦) ...
2306,2021/10/09,dufwjddufwjd,@ace27567 @hhans9215 자기의 권한을 명확하게 법의 태두리 안에서 잘...
2307,2021/10/07,naloongsbaby,"재인룽 감사룽,..,. 청년정책 마지막까지 잘 뽑아먹고 퇴장합니다…"
2308,2021/10/04,drewstar___,"오늘 신문 요약/ 청년 없는 청년 정책, 달에서 벌어지는 미국/중국 우주패권 정책,..."


In [259]:
#전처리 함수

def clean_str(text, special = False):
    # 리트윗 제거
    pattern = 'RT @[\w_]+: '
    text = re.sub(pattern=pattern, repl='', string=text)
    # 타계정 멘션 제거
    pattern = '@[\w_]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    # 해시태그 제거 
    pattern = '#'
    text = re.sub(pattern=pattern, repl='', string=text)
    # E-mail 제거
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # URL 제거 
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern=pattern, repl='', string=text)
    #. ? ! 등의 반복 제거
    pattern = r'\.+'
    text = re.sub(pattern=pattern, repl='.', string=text)
    pattern = r'\?+'
    text = re.sub(pattern=pattern, repl='?', string=text)
    pattern = r'\!+'
    text = re.sub(pattern=pattern, repl='!', string=text)
    pattern = r'\!~'
    text = re.sub(pattern=pattern, repl='~', string=text)
    pattern = r'\!,'
    text = re.sub(pattern=pattern, repl=',', string=text)
    # 한글 자음, 모음 제거
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # HTML 태그 제거
    pattern = '<[^>]*>'        
    text = re.sub(pattern=pattern, repl='', string=text)
    # 특수기호 제거
    if special:
        pattern = '[^\w\s\n]'
        text = re.sub(pattern=pattern, repl='', string=text)

        text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', string=text)
#         text = re.sub('\n', '.', string=text)
    return text.strip()

In [260]:
# 문장 단위 semantic을 기준으로 임베딩할 것이므로 \n \t 등을 모두 공백으로 처리
pattern_whitespace = re.compile(f'[{whitespace}]+')

# 기사 본문 공백 처리
twitter_df_n['내용_n1'] = twitter_df_n['내용'].str.replace(pattern_whitespace, ' ')
twitter_df_n['내용_n1'] = twitter_df_n['내용_n1'].apply(lambda x : unicodedata.normalize('NFC', x)).str.strip()

In [261]:
#version1 : 1) sns 데이터 전처리, 2) 문장 분리

#전처리
twitter_df_n['내용_n1'] = twitter_df_n['내용_n1'].apply(lambda x : clean_str(x,special = False))

#문장 분리
twitter_df_n['내용_n1'] = twitter_df_n['내용_n1'].progress_apply(lambda x: split_sentences(x))

100%|██████████████████████████████████████| 2310/2310 [00:05<00:00, 430.25it/s]


In [262]:
#version2 : 1) sns 데이터 전처리, 2) 문장 분리, 3)분리된 문장의 특수 문자 제거

#문장 분리 후 특수 문자 완전 제거
twitter_df_n['내용_n2'] = twitter_df_n['내용_n1'].apply(lambda x : [clean_str(sent,special = True) for sent in x])

# 전처리 결과 저장

In [263]:
store_path = './'
twitter_df_n.to_excel(store_path + '청년정책_twitter_preprocessed_v0.1.xlsx')